In [ ]:
import pandas as pd
import numpy as np
# from google.colab import files
# files.upload()

In [ ]:
Data = pd.read_csv('Dinos.txt',names=['Name'])
Data.head()


In [ ]:
Names = Data.Name

In [ ]:
NamesSep = Names.map(lambda x: list(x))
NamesSep.head()

In [ ]:
NamesSepWithStartStop = NamesSep.map(lambda x: x+['#'])
NamesSepWithStartStop.head()

In [ ]:
AllNames = pd.concat([NamesSepWithStartStop,NamesSepWithStartStop.map(lambda x: len(x))],axis=1)
AllNames.columns = ['Name','Length']

In [ ]:
AllNames.describe()

In [ ]:
TempNames = list(AllNames.Name)

In [ ]:
allChars = [char for name in TempNames for char in name]

In [ ]:
chars = list(set(allChars))
chars

In [ ]:
len(chars)

In [ ]:
char_to_token = dict(zip(chars,[num for num in range(len(chars))]))
char_to_token

In [ ]:
max_token = max(list(char_to_token.values()))
print(max_token)
token_to_encode = dict()
for i in char_to_token.values():
    temp = np.zeros((1,max_token+1))
    temp[:,i] = 1
    token_to_encode[i] = temp

token_to_encode

In [ ]:
def charTokenizer(characters):
    tokens = list()
    for i in range(len(characters)):
        tokens.append(char_to_token[characters[i]])
    return tokens

TestTokens = charTokenizer(TestName)
TestTokens

In [ ]:
def tokenEncoder(tokens):
    encoded = np.empty((len(tokens),max_token+1))
    for i in range(len(tokens)):
        encoded[i,:] = token_to_encode[tokens[i]]
    return encoded

In [ ]:
TestEncodings = tokenEncoder(TestTokens)
TestEncodings

In [ ]:
token_to_char = {value: key for key, value in char_to_token.items() }
token_to_char

In [ ]:
def encodingCharacterizer(encoding):
    characters = list()
    for i in range(len(encoding)):
        characters.append(token_to_char[encoding[i].argmax()])
    return characters

In [ ]:
encodingCharacterizer(TestEncodings)

In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
dtype = torch.FloatTensor
input_size, hidden_size, output_size = max_token+1+max_token+1, max_token+1, max_token+1

In [ ]:
w1 = torch.FloatTensor(np.random.randn(input_size, hidden_size)).type(dtype)
w1 =  Variable(w1, requires_grad=True)
w2 = torch.FloatTensor(np.random.randn(hidden_size, output_size)).type(dtype)
w2 = Variable(w2, requires_grad=True)
b1 = torch.FloatTensor(np.random.randn(1, hidden_size)).type(dtype)
b1 =  Variable(b1, requires_grad=True)
b2 = torch.FloatTensor(np.random.randn(1, output_size)).type(dtype)
b2 = Variable(b2, requires_grad=True)
w3 = torch.FloatTensor(np.random.randn(output_size, output_size)).type(dtype)
w3 = Variable(w3, requires_grad=True)
b3 = torch.FloatTensor(np.random.randn(1, output_size)).type(dtype)
b3 = Variable(b3, requires_grad=True)


In [ ]:
TestTensor = torch.FloatTensor(tokenEncoder(charTokenizer(AllNames.Name[2])))
TestTensor.size()

In [ ]:
def cycle(hidden,inputs):
    h_t = torch.tanh(torch.mm(torch.cat([inputs,hidden],dim=1),w1)+b1)
    out = torch.mm(torch.tanh(torch.mm(h_t,w2)+b2),w3)+b3
    return h_t, out
    
def forward(X,init=None):
    lenSeq = X.size(0)
    hiddens = []
    outs = []
    if init==None:
        h_t=torch.zeros(1,X.size(1))
    else:
        h_t = init
    for i in range(lenSeq):
        h_t,out = cycle(h_t,X[[i]])
        hiddens.append(h_t)
        outs.append(out)
    hiddenSeq = torch.cat(hiddens)
    outSeq = torch.cat(outs)
    return hiddenSeq,outSeq

In [ ]:
optimizer = torch.optim.Adam([w1,w2,w3,b1,b2,b3], lr = 0.001)

def backward(OutputSequence,TrueLabels):
    _, targets = TrueLabels.max(dim=1)
    #print(targets)
    loss = torch.nn.CrossEntropyLoss()(OutputSequence,targets)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    #print(loss.data)
    return loss

In [ ]:
hid,out = forward(TestTensor[0:-1])
backward(out,TestTensor[1:])

In [ ]:
Names = AllNames.Name

In [ ]:
EncodedNames = Names.apply(lambda x: tokenEncoder(charTokenizer(x)))

In [ ]:
np.array(EncodedNames.sample().tolist()[0]).shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
optimizer = torch.optim.Adam([w1,w2,w3,b1,b2,b3], lr = 0.001)
epochs = 200000
losses = []
for i in range(epochs):
    Sample = np.array(EncodedNames.sample().tolist()[0])
    X = torch.FloatTensor(Sample)
    hid,out = forward(X[0:-1])
    lossVal = backward(out,X[1:])
    losses.append(lossVal)
    if (i+1)%(epochs/100)==0:
        print('Epochs: {}/{}'.format(i+1,epochs))
    
plt.plot(losses)

In [ ]:
def generate(startLetter=None,sillyFactor=0.25):
    # if no start letter provided, pick one at random
    if startLetter==None:
        startLetter=token_to_char[round(np.random.rand()*max_token)]
    if startLetter=='#':
        return None
    generated = [startLetter]
    start = torch.FloatTensor(tokenEncoder(charTokenizer([startLetter])))
    h_t = torch.randn(1,hidden_size)*sillyFactor
    h_t,out = cycle(h_t,start)
    out = torch.softmax(out,dim=1)
    ## from one hot encoded output, back to the character
    _, pos = out.max(dim=1)
    out = torch.zeros(1,hidden_size)
    out[:,pos]=1
    # if the generated character is the stop token, return None
    if token_to_char[int(pos)] == '#':
        return None
    generated.append(token_to_char[int(pos)])
    # do a for loop and get an output until a stop token or 
    # max number of iterations
    for i in range(24):
        h_t,out = cycle(h_t,out)
        out = torch.softmax(out,dim=1)
        _, pos = out.max(dim=1)
        out = torch.zeros(1,hidden_size)
        out[:,pos]=1
        if token_to_char[int(pos)] == '#':
            break
        generated.append(token_to_char[int(pos)])
    #print(''.join(generated))
    return ''.join(generated)

In [ ]:
nonSense = [generate(startLetter=None,sillyFactor=np.random.rand()) for i in range(100)]

In [ ]:
nonSense

In [ ]:
[generate(startLetter='a',sillyFactor=0.1) for i in range(100)]